In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"
# 检查 GPU 是否可用  
import torch
if torch.cuda.is_available():  
   print("GPU 可用")  
   # 获取 GPU 设备数量  
   num_gpus = torch.cuda.device_count()  
   print(f"共有{num_gpus}个 GPU 设备")
   # 获取 GPU 设备信息  
   for i in range(num_gpus):  
       print(f"设备{i}: {torch.cuda.get_device_properties(i)}")  
else:  
   print("GPU 不可用")  
import sys
# sys.path.append("/workspace/notebooks/")
# sys.path.append("/workspace/notebooks/FSS/")
import numpy as np
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU 可用
共有1个 GPU 设备
设备0: _CudaDeviceProperties(name='NVIDIA GeForce RTX 2080 Ti', major=7, minor=5, total_memory=11011MB, multi_processor_count=68)


# Bert model
pip install transformers

Experimental results of bert-base-cased on GLUE 
| Task  | Metric                       | Result      | Training time | Result (FP16) | Training time (FP16) |
|-------|------------------------------|-------------|---------------|---------------|----------------------|
| CoLA  | Matthews corr                | 56.53       | 3:17          | 56.78         | 1:41                 |
| SST-2 | Accuracy                     | 92.32       | 26:06         | 91.74         | 13:11                |
| MRPC  | F1/Accuracy                  | 88.85/84.07 | 2:21          | 88.12/83.58   | 1:10                 |
| STS-B | Pearson/Spearman corr.       | 88.64/88.48 | 2:13          | 88.71/88.55   | 1:08                 |
| QQP   | Accuracy/F1                  | 90.71/87.49 | 2:22:26       | 90.67/87.43   | 1:11:54              |
| MNLI  | Matched acc./Mismatched acc. | 83.91/84.10 | 2:35:23       | 84.04/84.06   | 1:17:06              |
| QNLI  | Accuracy                     | 90.66       | 40:57         | 90.96         | 20:16                |
| RTE   | Accuracy                     | 65.70       | 57            | 65.34         | 29                   |
| WNLI  | Accuracy                     | 56.34       | 24            | 56.34         | 12                   |

# Training Bert with PyTorch

## 1、Get datasets and tokenizer, processing datasets

In [1]:
# pip install datasets transformers
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

ConnectionError: Couldn't reach 'glue' on the Hub (ConnectTimeout)

In [3]:
raw_datasets = load_dataset("glue", "mrpc")

ConnectionError: Couldn't reach 'glue' on the Hub (ConnectTimeout)